In [1]:
#Importing required libraries

import os
import numpy as np
import pandas as pd

In [2]:
data_dir = '20_newsgroups/' #will be used to access the directory containing the data
folders = sorted(os.listdir(os.path.join(data_dir))) #folders is a (sorted) list of the folder names in the data

if '.DS_Store' in folders:
    del folders[0]
# used it to remove '.DS_Store' item at folders[0] (as list is sorted, this file appears at 0 index due to its name) 
# when code is run on MacOS (not required for windows)

## Creating Data Dictionary

In this cell, *each file in each folder is opened, read and its contents are added to data as a string*
where data is a dictionary of the form { folder1 : \[doc1,doc2,....,doc1000\] , folder2 : \[doc1,doc2,doc3,....\] ..,folder20:\[..\]}
i.e. doc1 is a string containing entire contents of the first file in folder1, doc2 is a string containing contents 
of the second file in folder1 and so on.



In [3]:
data = {}

for folder in folders:
    data[folder] = []
    for file in os.listdir(os.path.join(data_dir,folder)):
        with open(os.path.join(data_dir,folder,file),encoding='latin-1') as opened_file:
            data[folder].append(opened_file.read())


In [ ]:
# x = 0
# for i in range(len(folders)):
#     y = len((data[folders[i]]))
#     print(folders[i], str(y))
#     x+=y
# print(x)

#prints no of files in each folder and finally the total no of files in all 20 folders (x).

In [ ]:
# import nltk
# nltk.download('stopwords')

## Creating stop words

Stop words will be ignored when encountered in the files as they are read. Here I've used stop words from nltk library, a file of common stop words in english that I found online and added to it punctuations from string library, and a few stop words of my own after looking at a file to see which phrases would best be ignored. Finally, I obtained a list of stop words that was 513 words long.

In [4]:
from nltk.corpus import stopwords
from string import punctuation #punctuation is a string made up all punctuations


stop = pd.read_csv("stop_words.csv",header=None) #stop_words.csv is a .csv file containing common stop words in english
stop = stop.values.tolist() #to convert pandas dataframe to a list (2D list with one column)

punctuations = list(punctuation) #to convert string punctuation into a list of strings as ['!','#'..])
stopWords = stopwords.words('english')  #stopWords is a list of 179 stop words from nltk 


stopWords += punctuations  #adding punctuations to list of stop words
stopWords+=['subject:','from:', 'date:', 'newsgroups:', 'message-id:', 'lines:', 'path:', 'organization:', 
            'would', 'writes:', 'references:', 'article', 'sender:', 'nntp-posting-host:', 'people', 
            'university', 'think', 'xref:', 'cantaloupe.srv.cs.cmu.edu', 'could', 'distribution:', 'first', 
            'anyone','world', 'really', 'since', 'right', 'believe', 'still', 'keywords','expires','approved'
            ,'archive','name','to:','or:','telephone:','fax:','reply-to:',"that's",'followup-to:',"we're",
            "let's", "what's","max>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'","going","using",]
# adding some more stop words of my own


for item in stop: #adding words from csv file not already in stop words list
    word = item[0]
    if word not in stopWords:
        stopWords.append(word)

#len(stopWords) #list of stop words is 513 words long 

## Building the vocabulary

vocab is a dictionary that is built by going through all files in each folder, word by word. Every word that is of length 5 characters or longer and is not in the list of stop words, is added to vocab as a key, with its value being the total number of times that word has occured in all of the files.


In the next cell vocab is sorted in descending order of frequency of words.

In [5]:
vocab = {}
for folder in folders: #fast iteration over list of folders
    for doc in data[folder]: #loop over every file in a folder; doc is a string containing entire contents of the file
        for word in doc.split(): #splits string doc into a list of strings word wise
            if word.lower() not in stopWords and len(word)>=5:
                if word.lower() not in vocab: #new words not already present are added as key to vocab with value 1
                    vocab[word.lower()] = 1
                else:
                    vocab[word.lower()] +=1   #update or increment value of keys (words) already present in vocab
 

In [6]:
# Sort the dictionary based on frequency of each word in vocab
import operator
vocab=sorted(vocab.items(),key=operator.itemgetter(1),reverse=True)

# converts dictionary of key value pairs into a list of tuples as [('word': count) ..]
# operator.itemgetter(1) takes the value of the key at index 1 (the count of the word at index 0 of current tuple).
# vocab = sorted(vocab.items(), key=lambda x: x[1],reverse = True) can also be used to acheieve the same

## Creating list of features

The K=2000 most frequently occuring words are chosen as features. As vocab is sorted in descending order of frequency, the top 2000 words are picked up and added to list of features called feature_list.

Thus, feature_list is a list of strings which are the 2000 most frequently occuring words in the data.

In [7]:
# Choosing top 2000 vocab words as features
feature_list=[]
i = 0
K = 2000
for key in vocab:
    feature_list.append(key[0])
    i += 1
    if i == K:
        break       

## Creating X and Y

Where Y is a numpy array of output classes (folder name) corresponding to each data point (each file). Y contains 19997 items. 

df is a pandas dataframe of shape (19997,2000). It uses the chosen features as column labels, with each row corresponding to a file and the number of times a feature word appears in the file is recorded in the column of that feature.

In [8]:
import numpy as np

Y = []
for folder in folders:
    for doc in data[folder]:
        Y.append(folder)

Y = np.array(Y)

# This code is used to create the dataframe. It takes time to run. This cell can be skipped if pickle is loaded

In [ ]:
import pandas as pd

df = pd.DataFrame(columns = feature_list)
for folder in folders: #fast iteration over folders
    for file in os.listdir(os.path.join(data_dir,folder)): #loop over every file in the folder
        df.loc[len(df)] = np.empty(len(feature_list))  #add a new row for every file: #changed to np.empty from np.zeros
        with open(os.path.join(data_dir,folder,file),encoding='latin-1') as opened_file: #to open the file 
            for word in opened_file.read().split(): #to split string containing the contents of the file word wise, loops over every word
                if word.lower() in feature_list:    
                    df[word.lower()][len(df)-1] += 1  #df[feature][current_row]
                    #if the word is a feature, its count is updated in the column of the feature, in the current row

#### Pickling the dataframe

The code in the above cell took 45-50 minutes to run. Saved the dataframe so as to not run the code repeatedly.

In [ ]:
# df.to_pickle('newsgroup.pickle') #dataframe to pickle

In [9]:
# load pickle to kernel

import pandas as pd
df = pd.read_pickle('newsgroup.pickle') #to read the pickle into the kernel

#### Creating X

X is a numpy array of shape (19997, 2000).

In [10]:
X = df.values #to assign values of the dataframe to X, without index labels or column labels

## Dividing data into training and testing sets

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,random_state = 0)

In [12]:
X_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 5., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# In-built Naive Bayes in sklearn

 - __score obtained:__ 0.8506 
 - __average precision:__ 0.86

In [13]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train,Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [14]:
y_pred1 = clf.predict(X_test)  #y_pred1 contains values predicted by in-built naive bayes algorithm in sklearn

In [15]:
from sklearn.metrics import classification_report,confusion_matrix

#printing score, confusion matrix and classification report
print("Score: ",clf.score(X_test,Y_test))

Score:  0.8506


In [16]:
print("Confusion Matrix: ")
print()
print(confusion_matrix(y_pred1,Y_test))

Confusion Matrix: 

[[185   2   0   0   0   0   0   0   0   1   0   0   0   7   2   1   2  10
    7  52]
 [  0 198   9   7   4  11   2   1   0   0   0   1   4   1   5   0   1   0
    0   1]
 [  1   8 216   2   7  13   0   1   0   0   0   2   6   0   1   0   0   0
    0   0]
 [  0   2   5 205   6   2   5   2   0   0   0   0   4   1   2   0   1   0
    2   1]
 [  0   7   2   8 208   0   4   1   0   0   1   1   3   0   0   0   0   0
    0   0]
 [  0   5   8   3   0 206   1   0   2   0   0   1   1   2   0   0   0   0
    0   0]
 [  0   5   2   7   4   0 230   6   5   2   1   0   2   1   1   0   3   2
    2   1]
 [  1   5   2   2   2   0   4 239   7   2   0   2   6   2   2   0   2   2
    3   1]
 [  0   4   0   1   1   0   2   4 265   0   2   0   1   4   3   0   1   3
    8   0]
 [  1   1   0   0   0   2   0   0   0 243   0   1   0   0   1   0   0   0
    2   4]
 [  1   1   0   0   0   0   1   1   0   0 224   0   0   2   1   0   1   0
    1   1]
 [  0   2   0   1   0   2   1   0   0   0   0

In [17]:
print("Classification report: ")
print(classification_report(y_pred1,Y_test))

Classification report: 
                          precision    recall  f1-score   support

             alt.atheism       0.79      0.69      0.74       269
           comp.graphics       0.78      0.81      0.80       245
 comp.os.ms-windows.misc       0.87      0.84      0.85       257
comp.sys.ibm.pc.hardware       0.85      0.86      0.86       238
   comp.sys.mac.hardware       0.88      0.89      0.88       235
          comp.windows.x       0.86      0.90      0.88       229
            misc.forsale       0.88      0.84      0.86       274
               rec.autos       0.89      0.84      0.86       284
         rec.motorcycles       0.93      0.89      0.91       299
      rec.sport.baseball       0.98      0.95      0.97       255
        rec.sport.hockey       0.97      0.96      0.96       234
               sci.crypt       0.93      0.93      0.93       233
         sci.electronics       0.87      0.85      0.86       252
                 sci.med       0.89      0.90      

# Method 2: Naive Bayes code from scratch: 


 - Average precision: 0.86.

### Fit Function

Creates a dictionary result as:
result = { 'total_data' : 14997, 'class1' : { 'total_count_datapoints" : total no of datapoints belonging to class1, 'total_count' : total no of feature words belonging to class1, 'feature1' : no of times feature1 appears in all data points belonging to class1, 'feature2' : count of appearance of feature2 in class 1,..},  class2' : {'total count': value, 'feature1': value,...}...
}

These counts are used to calculate (logarithmic) probabilities and hence determine output classes using Bayes theorem, assuming features to be independent of each other (Naive assumption).

This dictionary is used by functions called by predict function to calculate probabilties and hence decide output classes.

In [18]:
def fit(X_train,Y_train):
    result = {}
    result["total_data"] = len(Y_train) #count of total number of data points (in training set)
    class_values = set(Y_train) #set of all possible output classes
    
    for current_class in class_values: #iterate over every possible class
        result[current_class] = {} #value for key current_class is an (empty) dictionary
        current_class_rows = (Y_train == current_class) #a true/false array with true values for rows that have output class equal to current_class
        X_train_current = X_train[current_class_rows] #rows of X_train belonging to current_class
        Y_train_current = Y_train[current_class_rows ] #rows of Y_train belonging to current_Class
        result[current_class]["total_count_datapoints"] = len(Y_train_current)
        num_features = X_train.shape[1] #number of features in x train data (columns)
        total_words = 0
        for i in range(num_features): #iterate over features
            count_feature_in_class = X_train_current[:,i].sum()   #no of times the feature appears in all data points belonging to the current class
            result[current_class][feature_list[i]] = count_feature_in_class 
            total_words+= count_feature_in_class
        result[current_class]["total_count"]=total_words #total no of feature words in the class
    return result

In [19]:
def probability(dictionary,x,current_class):
    output = np.log(dictionary[current_class]['total_count_datapoints']) - np.log(dictionary["total_data"])#logarithmic class probability    for i in range(len(feature_list)):       #iterating over every feature
    for i in range(len(feature_list)):
        current_word_count = dictionary[current_class][feature_list[i]] + 1  #no of times feature appears in current_class. +1 for laplace correction
        total_word_count = dictionary[current_class]["total_count"]+len(feature_list)  #no of data points in current class with laplace correction
        current_word_probability = np.log(current_word_count) - np.log(total_word_count)
        if int(x[i])!= 0: #to ignore words having zero frequency
            output += current_word_probability #logarithmic probabilties added instead of being multiplied
    return output


In [20]:
def predictSinglePoint(dictionary,x):
    classes = dictionary.keys() #all possible output classes
    best_p = -1000
    best_class = -1
    first_run = True #for first run of loop
    for current_class in classes: #iterate over all classes
        if current_class == "total_data": #to skip total_data key as it doesnt correspond to an output class
            continue
        p_current_class = probability(dictionary,x,current_class) #probability of current class being the class of the data point
        if (first_run or p_current_class > best_p):
            best_p = p_current_class
            best_class = current_class
        first_run = False
   
    return best_class

# The probability of the data point belonging to each class is calculated. The class with the highest probability is chosen
# to be the output class for the data point

In [21]:
def predict(dictionary,X_test):
    y_pred = []
    for x in X_test: #iterate over ever row or data point
        x_class = predictSinglePoint(dictionary,x)
        y_pred.append(x_class)
    
    return y_pred 

In [22]:
dictionary = fit(X_train,Y_train)

# Predict function takes time to run. This cell can be skipped. proceed by loading the pickle

In [ ]:
y_pred = predict(dictionary,X_test)

In [ ]:
# pickled the output as predict function took time to run
# import pickle
# with open('predictions.pickle', 'wb') as f:
#     pickle.dump(y_pred, f)

In [23]:
import pickle

with open('predictions.pickle','rb') as f:
    y_pred = pickle.load(f)

In [24]:
print("Confusion Matrix: ")
print()
print(confusion_matrix(y_pred,Y_test))

Confusion Matrix: 

[[184   1   0   0   0   1   0   0   0   0   0   0   0   6   2   0   2  16
    8  51]
 [  0 204   9   5   4  10   3   2   0   0   0   2   2   0   5   0   0   0
    2   1]
 [  1   5 214   2   4  12   1   0   0   0   0   1   5   0   1   0   0   0
    0   0]
 [  0   1   5 207   4   2   3   2   0   0   0   0   2   1   1   0   1   0
    2   1]
 [  0   9   3   6 212   0   4   0   0   0   1   1   2   0   1   0   0   0
    0   0]
 [  0   4   7   2   1 208   1   0   2   0   0   2   0   2   0   0   0   0
    0   0]
 [  0   7   3   7   4   1 229   7   6   1   0   0   2   1   1   0   0   2
    2   1]
 [  2   5   2   4   2   0   6 243   6   2   0   2   5   2   3   0   2   2
    2   3]
 [  0   3   0   2   3   0   2   6 267   0   2   0   0   6   4   0   1   3
    5   2]
 [  1   1   0   0   0   0   0   0   0 245   1   1   0   0   3   0   0   0
    2   1]
 [  1   1   0   0   0   0   1   1   0   0 223   0   0   1   1   0   1   0
    1   1]
 [  0   4   1   2   0   1   1   0   0   0   0

In [25]:
print("Classification Report: ")
print(classification_report(y_pred,Y_test))

Classification Report: 
                          precision    recall  f1-score   support

             alt.atheism       0.79      0.68      0.73       271
           comp.graphics       0.81      0.82      0.81       249
 comp.os.ms-windows.misc       0.86      0.87      0.86       246
comp.sys.ibm.pc.hardware       0.86      0.89      0.88       232
   comp.sys.mac.hardware       0.90      0.89      0.89       239
          comp.windows.x       0.87      0.91      0.89       229
            misc.forsale       0.88      0.84      0.86       274
               rec.autos       0.90      0.83      0.86       293
         rec.motorcycles       0.94      0.87      0.91       306
      rec.sport.baseball       0.99      0.96      0.97       255
        rec.sport.hockey       0.97      0.96      0.96       232
               sci.crypt       0.94      0.92      0.93       237
         sci.electronics       0.91      0.87      0.89       254
                 sci.med       0.88      0.93      

## Conclusion

The Naive Bayes algorithm that was implemented from scratch was found to have an average precision of 0.86 which is the same as the sklearn multinomial Naive Bayes algorithm. The in-built sklearn implementation had coefficient of determination (score) = 0.8506